In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("../data/raw/raw_settlement_data.csv")
df
# We predict:

# Will this settlement be completed within 3 days?

,user_id,amount,created_at,settled_at,is_settled
0,1,185.28,2024-04-09,2024-04-11,1
1,1,377.87,2024-09-14,2024-09-15,1
2,1,255.90,2024-11-09,2024-11-09,1
3,1,269.11,2024-02-28,2024-03-01,1
4,1,1091.29,2024-02-24,2024-02-28,1
...,...,...,...,...,...
8071,300,583.53,2024-09-16,2024-09-18,1
8072,300,307.22,2024-06-29,2024-07-01,1
8073,300,447.35,2024-11-19,2024-11-21,1
8074,300,400.28,2024-12-08,2024-12-08,1


In [3]:

df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
df["settled_at"] = pd.to_datetime(df["settled_at"], errors="coerce")
df["settled_at"] = (
    df["settled_at"]
    .dt.tz_localize("UTC")
    .dt.tz_convert("Asia/Kolkata")
)
df["created_at"] = (
    df["created_at"]
    .dt.tz_localize("UTC")
    .dt.tz_convert("Asia/Kolkata")
)


In [4]:
df.dtypes

user_id                              int64
amount                             float64
created_at    datetime64[us, Asia/Kolkata]
settled_at    datetime64[us, Asia/Kolkata]
is_settled                           int64
dtype: object

In [5]:
df['delay_days'] = (df['settled_at'] - df['created_at']).dt.days
df

,user_id,amount,created_at,settled_at,is_settled,delay_days
0,1,185.28,2024-04-09 05:30:00+05:30,2024-04-11 05:30:00+05:30,1,2
1,1,377.87,2024-09-14 05:30:00+05:30,2024-09-15 05:30:00+05:30,1,1
2,1,255.90,2024-11-09 05:30:00+05:30,2024-11-09 05:30:00+05:30,1,0
3,1,269.11,2024-02-28 05:30:00+05:30,2024-03-01 05:30:00+05:30,1,2
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4
...,...,...,...,...,...,...
8071,300,583.53,2024-09-16 05:30:00+05:30,2024-09-18 05:30:00+05:30,1,2
8072,300,307.22,2024-06-29 05:30:00+05:30,2024-07-01 05:30:00+05:30,1,2
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0


In [6]:
df = df.sort_values(by=['user_id', 'created_at'], ascending=[True, True])

In [7]:
import numpy as np
df['delayed_flag'] = np.where(df['delay_days'] > 3, 1,0)

df["past_total_settlements"] = df.groupby("user_id").cumcount()
df

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4
...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32


In [8]:
df['past_delayed_settlements'] = (
    df.groupby("user_id")["delayed_flag"]
        .cumsum()
        .shift(1)
        .fillna(0)
)


In [9]:
df['past_delayed_settlements'] = df['past_delayed_settlements'].astype(int)

In [11]:
df['past_delay_rate'] = (df['past_delayed_settlements'] / df['past_total_settlements'])
df["past_delay_rate"] = df["past_delay_rate"].fillna(0)


In [58]:
df.to_pickle("../data/processed/featurev1.pkl")

In [59]:
df_1 = pd.read_pickle("../data/processed/featurev1.pkl")
df_1

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements,past_delayed_settlements,past_delay_rate
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0,0,0.000000
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1,0,0.000000
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2,0,0.000000
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3,1,0.333333
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4,1,0.250000
...,...,...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29,4,0.137931
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30,4,0.133333
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31,4,0.129032
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32,4,0.125000


In [60]:
df_1['delay_days_overtime'] = (
    df_1.groupby("user_id")["delay_days"]
        .cumsum()
        .shift(1)
        .fillna(0)
)
df_1['delay_days_overtime'] = df_1['delay_days_overtime'].astype(int)



In [61]:
df_1['avg_settlement_time'] = (df_1['delay_days_overtime'] / df_1['past_total_settlements'])
df_1['avg_settlement_time'] = df_1['avg_settlement_time'].fillna(0)

In [62]:
df_1['avg_amount'] = (
    df_1.groupby("user_id")["amount"]
        .cumsum()
        .shift(1)
        .fillna(0)
)
df_1

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements,past_delayed_settlements,past_delay_rate,delay_days_overtime,avg_settlement_time,avg_amount
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0,0,0.000000,0,0.000000,0.00
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1,0,0.000000,2,2.000000,658.34
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2,0,0.000000,4,2.000000,857.77
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3,1,0.333333,14,4.666667,1124.66
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4,1,0.250000,15,3.750000,1454.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29,4,0.137931,59,2.034483,13029.85
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30,4,0.133333,61,2.033333,13477.20
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31,4,0.129032,61,1.967742,13877.48
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32,4,0.125000,61,1.906250,14363.41


In [63]:
df_1['past_avg_amount'] = (df_1['avg_amount']/df_1['past_total_settlements'])
df_1['past_avg_amount'] = df_1['past_avg_amount'].fillna(0)
df_1

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements,past_delayed_settlements,past_delay_rate,delay_days_overtime,avg_settlement_time,avg_amount,past_avg_amount
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0,0,0.000000,0,0.000000,0.00,0.000000
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1,0,0.000000,2,2.000000,658.34,658.340000
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2,0,0.000000,4,2.000000,857.77,428.885000
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3,1,0.333333,14,4.666667,1124.66,374.886667
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4,1,0.250000,15,3.750000,1454.10,363.525000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29,4,0.137931,59,2.034483,13029.85,449.305172
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30,4,0.133333,61,2.033333,13477.20,449.240000
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31,4,0.129032,61,1.967742,13877.48,447.660645
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32,4,0.125000,61,1.906250,14363.41,448.856562


In [64]:
df_1['amount_vs_user_avg'] = (df_1['amount']/df_1['past_avg_amount'])
df_1

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements,past_delayed_settlements,past_delay_rate,delay_days_overtime,avg_settlement_time,avg_amount,past_avg_amount,amount_vs_user_avg
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0,0,0.000000,0,0.000000,0.00,0.000000,inf
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1,0,0.000000,2,2.000000,658.34,658.340000,0.302929
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2,0,0.000000,4,2.000000,857.77,428.885000,0.622288
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3,1,0.333333,14,4.666667,1124.66,374.886667,0.878772
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4,1,0.250000,15,3.750000,1454.10,363.525000,3.001967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29,4,0.137931,59,2.034483,13029.85,449.305172,0.995648
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30,4,0.133333,61,2.033333,13477.20,449.240000,0.891016
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31,4,0.129032,61,1.967742,13877.48,447.660645,1.085487
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32,4,0.125000,61,1.906250,14363.41,448.856562,0.339685


In [66]:
df_1['avg_delay_last_5'] = (
    df_1.groupby("user_id")['delay_days']
    .rolling(window=5, min_periods=1)
    .mean()
    .shift(1)
    .reset_index(level=0, drop=True)
)
df_1
df_1['avg_delay_last_5']= df_1['avg_delay_last_5'].fillna(0)

In [67]:
import numpy as np

df_1.replace([np.inf, -np.inf], 0, inplace=True)
df_1

,user_id,amount,created_at,settled_at,is_settled,delay_days,delayed_flag,past_total_settlements,past_delayed_settlements,past_delay_rate,delay_days_overtime,avg_settlement_time,avg_amount,past_avg_amount,amount_vs_user_avg,avg_delay_last_5
17,1,658.34,2024-01-05 05:30:00+05:30,2024-01-07 05:30:00+05:30,1,2,0,0,0,0.000000,0,0.000000,0.00,0.000000,0.000000,0.000000
12,1,199.43,2024-01-09 05:30:00+05:30,2024-01-11 05:30:00+05:30,1,2,0,1,0,0.000000,2,2.000000,658.34,658.340000,0.302929,2.000000
20,1,266.89,2024-02-05 05:30:00+05:30,2024-02-15 05:30:00+05:30,1,10,1,2,0,0.000000,4,2.000000,857.77,428.885000,0.622288,2.000000
7,1,329.44,2024-02-19 05:30:00+05:30,2024-02-20 05:30:00+05:30,1,1,0,3,1,0.333333,14,4.666667,1124.66,374.886667,0.878772,4.666667
4,1,1091.29,2024-02-24 05:30:00+05:30,2024-02-28 05:30:00+05:30,1,4,1,4,1,0.250000,15,3.750000,1454.10,363.525000,3.001967,3.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,300,447.35,2024-11-19 05:30:00+05:30,2024-11-21 05:30:00+05:30,1,2,0,29,4,0.137931,59,2.034483,13029.85,449.305172,0.995648,1.200000
8074,300,400.28,2024-12-08 05:30:00+05:30,2024-12-08 05:30:00+05:30,1,0,0,30,4,0.133333,61,2.033333,13477.20,449.240000,0.891016,1.200000
8065,300,485.93,2024-12-14 05:30:00+05:30,2024-12-14 05:30:00+05:30,1,0,0,31,4,0.129032,61,1.967742,13877.48,447.660645,1.085487,1.200000
8059,300,152.47,2024-12-25 05:30:00+05:30,2024-12-25 05:30:00+05:30,1,0,0,32,4,0.125000,61,1.906250,14363.41,448.856562,0.339685,0.800000


In [69]:
df_1 = df_1.drop(columns=['user_id', 'created_at', 'settled_at', 'is_settled', 'delay_days', 'past_delayed_settlements', 'delay_days_overtime', 'avg_amount'])

In [73]:
df_1.to_pickle("../data/processed/final_dataset.pkl")